In [4]:
import tensorflow as tf
import tf2onnx
from tensorflow.keras.models import load_model, Model
import onnx

In [24]:
model=load_model('models/affectnet_emotions/mobilenet_7.h5')


In [9]:
input_signature = [tf.TensorSpec([1, 224, 224, 3], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature)

In [10]:
onnx.save(onnx_model, "mobnet.onnx")

In [11]:
import cv2

In [17]:
import onnxruntime as ort
import numpy as np

img = cv2.imread('test_images/0.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img.astype(np.float32)
img[..., 0] -= 103.939
img[..., 1] -= 116.779
img[..., 2] -= 123.68
input1 = np.expand_dims(img, axis=0)
# img.shape
# # Change shapes and types to match model
# input1 = np.zeros((1, 100, 100, 3), np.float32)

# # Start from ORT 1.10, ORT requires explicitly setting the providers parameter if you want to use execution providers
# # other than the default CPU provider (as opposed to the previous behavior of providers getting set/registered by default
# # based on the build flags) when instantiating InferenceSession.
# # Following code assumes NVIDIA GPU is available, you can specify other execution providers or don't include providers parameter
# # to use default CPU provider.
sess = ort.InferenceSession("mobnet.onnx", providers=["CPUExecutionProvider"])

# # Set first argument of sess.run to None to use all model outputs in default order
# # Input/output names are printed by the CLI and can be set with --rename-inputs and --rename-outputs
# # If using the python API, names are determined from function arg names or TensorSpec names.
results_ort = sess.run(["emotion_preds"], {"x": input1})

# import tensorflow as tf
# model = tf.saved_model.load("path/to/savedmodel")
# results_tf = model(input1)

# for ort_res, tf_res in zip(results_ort, results_tf):
#     np.testing.assert_allclose(ort_res, tf_res, rtol=1e-5, atol=1e-5)

# print("Results match")

In [25]:
model(input1)

<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
array([[0.00333036, 0.00210853, 0.02542588, 0.5138411 , 0.32844722,
        0.02245977, 0.10438712]], dtype=float32)>

In [26]:
results_ort

[array([[0.00333036, 0.00210852, 0.02542585, 0.5138409 , 0.32844752,
         0.02245977, 0.10438711]], dtype=float32)]

In [22]:
np.argmax(results_ort[0])

3

In [23]:
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
